# Get familiar with traces

#### Learning goals:
- Get familiar with the tracing format
- Learn how to write code and record traces
- Learn how to draw traces
- Learn how to access input data in the code

In [1]:
%load_ext autoreload
%autoreload 2

from cwtoolbox import CaptureDevice

## 1. The trace format

In [2]:
capture_device = CaptureDevice.create("CWLITEXMEGA")
capture_device.compile(code='asm("nop");')
capture_device.flash()

XMEGA Programming flash...
XMEGA Reading flash...
Verified flash OK, 2099 bytes


In [3]:
data = capture_device.capture(
    number_of_traces=1,
    number_of_samples=100,
    input=lambda _: [],
)

100%|██████████| 1/1 [00:00<00:00, 80.48it/s]


<div style="border: 3px solid plum; border-radius: 5px; padding: 5px; width: calc(100% - 20px);">
<div class="h2" style="font-variant: all-small-caps;">Exercise 1</div>

1. Which datatype is `data`? Which is `data[0]` Which is `data["trace"]`?
2. Calculate the maximum value of the whole trace. Calculate its absolute maximum value. Calculate the absolute maximum value of the first 50 trace points.

Hints:
- https://numpy.org/doc/stable/user/absolute_beginners.html#basic-array-operations
- https://numpy.org/doc/stable/user/absolute_beginners.html#indexing-and-slicing
</div>

<div style="border: 3px solid plum; border-radius: 5px; padding: 5px; width: calc(100% - 20px);">
<div class="h2" style="font-variant: all-small-caps;">Exercise 2</div>

1. Record 10 traces with the same C-code as above.
2. Create a plot displaying the 1st trace.
3. Create a plot which stacks the 1st and 2nd trace on top of each other. Create a single plot and shift the y-axis.
</div>


## 2. Different code and inputs

<div style="border: 3px solid plum; border-radius: 5px; padding: 5px; width: calc(100% - 20px);">
<div class="h2" style="font-variant: all-small-caps;">Exercise 3</div>

1. Write a C program which sums the first 10 natural numbers.
   Load it on the ChipWhisperer record a trace of that program.
2. Record a trace of a C program which sums the first 20 natural numbers.
3. Check by visual inspection that the traces differ.

Hints:
- The argument `code` in `capture_device.compile()` takes arbitrary C code.

</div>


In [9]:
import plotly.graph_objects as pgo

def run_source_and_trace(source):
    with open(source,'r') as f:
        c_source = f.read()
    print(f"Flashing:\n{c_source}")
    capture_device.compile(code=c_source)
    capture_device.flash()

    d = capture_device.capture(
        number_of_traces=1,
        number_of_samples=500,
        input=lambda _: [],
    )

    fig = pgo.Figure()
    fig.add_trace(pgo.Scatter(y=d["trace"][0]))
    fig.show()


In [10]:
run_source_and_trace('./add_first_10.c')

Flashing:
volatile int x = 0;
for(int i = 1; i <= 10; i++){
    x = x + i;
}

XMEGA Programming flash...
XMEGA Reading flash...
Verified flash OK, 2147 bytes


100%|██████████| 1/1 [00:00<00:00, 59.84it/s]


In [11]:
run_source_and_trace('./add_first_20.c')

Flashing:
volatile int x = 0;
for(int i = 1; i <= 20; i++){
    x = x + i;
}
XMEGA Programming flash...
XMEGA Reading flash...
Verified flash OK, 2147 bytes


100%|██████████| 1/1 [00:00<00:00, 57.06it/s]


<div style="border: 3px solid plum; border-radius: 5px; padding: 5px; width: calc(100% - 20px);">
<div class="h2" style="font-variant: all-small-caps;">Exercise 4</div>

1. Record a trace which sums the first $n$ natural numbers. $n$ shall be given from outside.
2. Compare different recordings.

Hints: 
- `input` is a function which is called once for each recorded trace. It returns a list of numbers which is converted into a byte array. This can be used in the C-code accessing `input` which has the type `uint8_t *`.
- The actual C program which is created during the call to `capture` can be found here: [./build/generic_simpleserial.c](./build/generic_simpleserial.c)

</div>

<div style="border: 3px solid plum; border-radius: 5px; padding: 5px; width: calc(100% - 20px);">
<div class="h2" style="font-variant: all-small-caps;">Exercise 5</div>

1. Use _one_ call to `capture` to record 10 traces with random $1 \leq n \leq 10$.
2. Plot all recorded traces on top of each other.
3. Plot all recorded traces on top of each other, sorted by $n$ ascending.

Hints:
- Use `random.randint(a, b)` to generate the random numbers. More info on random can be found here: [https://docs.python.org/3/library/random.html](https://docs.python.org/3/library/random.html)
- `numpy.argsort` might help.

</div>
